This is a how to guide for importing data from .csv file to Django database (SQLite). You will see example data classes. Please note that those classes are for demonstration only. You are free to create classes in the way you see fit the best. 

The end goal is to create a JSON data that will be used to import data by Django built-in command. The format we want is structured as below. (more detail check -> https://docs.djangoproject.com/en/5.1/howto/initial-data/)

```JSON
[
  {
    "model": "myapp.classname",
    "pk": 1,
    "fields": {
      "attribute_name1": "Value",
      "attribute_name2": "Value"
    }
  },
  {
    "model": "myapp.field",
    "pk": 2,
    "fields": {
      "field_id": "32",
      "field_name": "Corner"
    }
  }
]

```

In Python, you can assume that JSON is Dictionary-like data type. So, the overall step is .csv -> Pandas dataframe -> dictionary ->JSON. 

Let's start with importing necessary modules. In this case, we only need 2 modules. Then load data from .csv to Pandas dataframe (we did this before in lab 2).

In [1]:
# import python's module
import pandas as pd
import json

In [2]:
# load data
log_df = pd.read_csv("operation-log.csv")  
log_df
# _df stands for data frame. It is a common suffix to indicate variable type. 

,date,operator,location,operation,note,Power Unit,Seeds planted,Seeding Rate (seeds/ac),Fertilizers applied,Fertilizer Rate (lb/ac)
0,4/24/2022,Evan,200,Spread/Spray,2-4D round up burn down,Hagie STS12,NaN,NaN,NaN,NaN
1,4/27/2022,Bryan,6,Spread/Spray,"20oz ru, 11 24d",Hagie STS12,NaN,NaN,NaN,NaN
2,4/27/2022,Bryan,5,Spread/Spray,20 oz ru/ 11 oz 24d,Hagie STS12,NaN,NaN,NaN,NaN
3,4/27/2022,Bryan,Cotton,Spread/Spray,20 oz ru. 11 oz 24d,Hagie STS12,NaN,NaN,NaN,NaN
4,4/27/2022,Bryan,101,Spread/Spray,20 oz ru 11 oz 24d,Hagie STS12,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
428,11/17/2022,Ceres,105,Spread/Spray,VRF MAP: 153 lbs/ac\nVRF Potash: 133 lbs/ac,NaN,NaN,NaN,"MAP,Potash (0-0-60)",NaN
429,11/17/2022,Ceres,111,Spread/Spray,VRF MAP: 139 lbs/ac\nVRF Potash: 121 lbs/ac,NaN,NaN,NaN,"MAP,Potash (0-0-60)",NaN
430,11/17/2022,Ceres,33,Spread/Spray,VRF MAP: 154 lbs/ac,NaN,NaN,NaN,MAP,NaN
431,11/17/2022,Ceres,41,Spread/Spray,VRF MAP: 115.8 lbs/ac,NaN,NaN,NaN,MAP,NaN


First data class I want to create is Operator class ass below. As you can see that I created a class with attributes that we don't have data. That is okay, we can leave those attributes optional by adding `blank=True, null=True` (more detail here -> https://docs.djangoproject.com/en/5.1/ref/models/fields/#field-options).

```python 
class Operator(models.Model):
    firstname = models.CharField(max_length=30)
    lastname = models.CharField(max_length=30, blank=True, null=True)
    middlename = models.CharField(max_length=30, blank=True, null=True)
    phone = models.CharField(max_length=10, blank=True, null=True)
```

You can think of a database class as a table. So, let's create an `Operator` table (dataframe). We will use Pandas' Unique fucntion to see how many operator rows we need to create. (more detail about unique function -> https://pandas.pydata.org/docs/reference/api/pandas.unique.html)

In [3]:
log_df['operator'].unique() # find a unique value in dataframe's `operator` column

array(['Evan', 'Bryan', 'Aaron', 'Rachel', 'Ceres'], dtype=object)

The result from unique function is an array which is iterable. We will iterate over the array and create a dataframe. There are several ways to create a dataframe. But I recommend doing by using a list of dict or a dict of list. I will show you the first case. 

In [4]:
# create a list which we will add dictionaries later
operators = [] # suffix s is common way to indicate a list (array)
for person in log_df['operator'].unique(): # we iterate over the result from unique function
    operators.append( # append new element to the list
        { # curly bracket indicate the starting of dictionary
            "firstname": person, # each key-value pair
            "lastname": "",
            "phone": ""
        } # the end of dictionary
    )
operators # check the outcome

[{'firstname': 'Evan', 'lastname': '', 'phone': ''},
 {'firstname': 'Bryan', 'lastname': '', 'phone': ''},
 {'firstname': 'Aaron', 'lastname': '', 'phone': ''},
 {'firstname': 'Rachel', 'lastname': '', 'phone': ''},
 {'firstname': 'Ceres', 'lastname': '', 'phone': ''}]

Now we have a list of dictionaries. Making a dataframe is just one function. Note that I leave `lastname` and `phone` emtyp. This is an explicit way to work with optional attribute. You can also just ignore that attribute like I did with `middlename`. You will get the same outcome. 

In [5]:
# create a dataframe
operator_df = pd.DataFrame(operators)
operator_df

,firstname,lastname,phone
0,Evan,,
1,Bryan,,
2,Aaron,,
3,Rachel,,
4,Ceres,,


The next step is to structure the data as required (check data format above). We will need to add `model` which is the class's name in the database. And `pk` or primary key which is the row's index. We want JSON at the end, but JSON is pretty much like dictionary in Python. So, we just need to create a (list of) dictionary.

In [6]:
operators = [] # create an empty list
for index, row in operator_df.iterrows(): # we iterate over the dataframe row by row
    operators.append( # append new element to the list
        { # curly bracket indicate the starting of dictionary
            "model": "acrelog.operator",  # the class name
            "pk": index+1, # index starts at 0 while primary key starts at 1. 
            "fields": { # this is a dictionary of attributes
                "firstname": row['firstname'] # only need firstname for now
            }
        } # the end of dictionary
    )
operators # see the result

[{'model': 'acrelog.operator', 'pk': 1, 'fields': {'firstname': 'Evan'}},
 {'model': 'acrelog.operator', 'pk': 2, 'fields': {'firstname': 'Bryan'}},
 {'model': 'acrelog.operator', 'pk': 3, 'fields': {'firstname': 'Aaron'}},
 {'model': 'acrelog.operator', 'pk': 4, 'fields': {'firstname': 'Rachel'}},
 {'model': 'acrelog.operator', 'pk': 5, 'fields': {'firstname': 'Ceres'}}]

Now we have the data in the format that we need. To convert into JSON, you can just run the following command. Then you can copy the result and check with https://jsonlint.com/

In [7]:
json.dumps(operators)

'[{"model": "acrelog.operator", "pk": 1, "fields": {"firstname": "Evan"}}, {"model": "acrelog.operator", "pk": 2, "fields": {"firstname": "Bryan"}}, {"model": "acrelog.operator", "pk": 3, "fields": {"firstname": "Aaron"}}, {"model": "acrelog.operator", "pk": 4, "fields": {"firstname": "Rachel"}}, {"model": "acrelog.operator", "pk": 5, "fields": {"firstname": "Ceres"}}]'

In [8]:
# or write the JSON to the file directly 
with open('operator-data.json', 'w') as fp:
    json.dump(operators, fp=fp, indent=4)

You might wonder why we create a dictionary then convert into a dataframe just for turn it into a dictionary again. Can you just make a dictionary that follows the required stucture directly? Absolutely yes. In some cases, that will be the better way to deal with the data. But there is a usecase for a dataframe as well. You will see it soon. For now, let's create another data class. 

```python
class Operation(models.Model):
    date = models.DateField()
    note = models.CharField(max_length=300, blank=True, null=True)
    operator = models.ForeignKey(Operator, on_delete=models.CASCADE)
```
In this class, it has a foreignkey that points to Operator class. So, instead of recording operator name, we will keep the primary key that points to that operator. For better understanding, I will trim the original dataframe to keep only the data that we will play with.

In [9]:
# you can subset the dataframe with any columns (and order)
sub_log_df = log_df[['date', 'operator', 'note']]
sub_log_df

,date,operator,note
0,4/24/2022,Evan,2-4D round up burn down
1,4/27/2022,Bryan,"20oz ru, 11 24d"
2,4/27/2022,Bryan,20 oz ru/ 11 oz 24d
3,4/27/2022,Bryan,20 oz ru. 11 oz 24d
4,4/27/2022,Bryan,20 oz ru 11 oz 24d
...,...,...,...
428,11/17/2022,Ceres,VRF MAP: 153 lbs/ac\nVRF Potash: 133 lbs/ac
429,11/17/2022,Ceres,VRF MAP: 139 lbs/ac\nVRF Potash: 121 lbs/ac
430,11/17/2022,Ceres,VRF MAP: 154 lbs/ac
431,11/17/2022,Ceres,VRF MAP: 115.8 lbs/ac


We will use the map function (more detail -> https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html) to map from the operator's name to primary key. The map functions take several options. But we will use dictionary. The dictionary we want has operator's names as keys and primary keys as values. Do you feel that this structure looks familar. Yes, that is the operator dataframe that we created earlier. 

In [10]:
operator_df

,firstname,lastname,phone
0,Evan,,
1,Bryan,,
2,Aaron,,
3,Rachel,,
4,Ceres,,


As I mentioned earlier, dataframe is made from a list of dictionaries or a dictionary of lists. We can convert it back as well.

In [11]:
operator_df.to_dict()

{'firstname': {0: 'Evan', 1: 'Bryan', 2: 'Aaron', 3: 'Rachel', 4: 'Ceres'},
 'lastname': {0: '', 1: '', 2: '', 3: '', 4: ''},
 'phone': {0: '', 1: '', 2: '', 3: '', 4: ''}}

The format you see above is a dictionary of list. We want to use firstname to map with primary key (or dataframe's index). So we will only care about the `firstname`.

In [12]:
operator_df.to_dict()['firstname']

{0: 'Evan', 1: 'Bryan', 2: 'Aaron', 3: 'Rachel', 4: 'Ceres'}

Almost done. We want a dictionary that keys are firstname and values are primary keys. But what we got is opposite. No problem, we just need to inverse it (and add 1 to pandas' index to make it become primary key)

In [13]:
index_name = operator_df.to_dict()['firstname']
name2pk = dict((v, k+1) for k, v in index_name.items())
name2pk

{'Evan': 1, 'Bryan': 2, 'Aaron': 3, 'Rachel': 4, 'Ceres': 5}

We got it. Now, let map the operator's name to primary key.

In [14]:
sub_log_df['operator'] = sub_log_df['operator'].map(name2pk) 
sub_log_df

/var/folders/_t/xf80sxmn3c99xrwsspdr3tc00000gp/T/ipykernel_27436/2883430231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_log_df['operator'] = sub_log_df['operator'].map(name2pk)


,date,operator,note
0,4/24/2022,1,2-4D round up burn down
1,4/27/2022,2,"20oz ru, 11 24d"
2,4/27/2022,2,20 oz ru/ 11 oz 24d
3,4/27/2022,2,20 oz ru. 11 oz 24d
4,4/27/2022,2,20 oz ru 11 oz 24d
...,...,...,...
428,11/17/2022,5,VRF MAP: 153 lbs/ac\nVRF Potash: 133 lbs/ac
429,11/17/2022,5,VRF MAP: 139 lbs/ac\nVRF Potash: 121 lbs/ac
430,11/17/2022,5,VRF MAP: 154 lbs/ac
431,11/17/2022,5,VRF MAP: 115.8 lbs/ac


Before we make the final dictionary, we need to deal with date format. Django's SQLite expects `YYYY-MM-DD` format. So, we need to fix our date column a little bit. What you see is only string (a series of charactor). Python has a date (or datetime) variable for dealing with this type of data. Date data type is numerial. So, we can do calulation and also, in our case, format in the way we want. First, we need to convert string to datetime. 

In [15]:
pd.to_datetime(sub_log_df['date'])

0     2022-04-24
1     2022-04-27
2     2022-04-27
3     2022-04-27
4     2022-04-27
         ...    
428   2022-11-17
429   2022-11-17
430   2022-11-17
431   2022-11-17
432   2022-11-17
Name: date, Length: 433, dtype: datetime64[ns]

As you can see, we have a date column which we can do many thing on it. But we will just convert it back into string (with the format that we want). Then we will make a dictionary and JSON. The function we use is `strftime` (string from time).

In [16]:
sub_log_df['date'] = pd.to_datetime(sub_log_df['date']).dt.strftime('%Y-%m-%d')
sub_log_df

/var/folders/_t/xf80sxmn3c99xrwsspdr3tc00000gp/T/ipykernel_27436/2137056224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_log_df['date'] = pd.to_datetime(sub_log_df['date']).dt.strftime('%Y-%m-%d')


,date,operator,note
0,2022-04-24,1,2-4D round up burn down
1,2022-04-27,2,"20oz ru, 11 24d"
2,2022-04-27,2,20 oz ru/ 11 oz 24d
3,2022-04-27,2,20 oz ru. 11 oz 24d
4,2022-04-27,2,20 oz ru 11 oz 24d
...,...,...,...
428,2022-11-17,5,VRF MAP: 153 lbs/ac\nVRF Potash: 133 lbs/ac
429,2022-11-17,5,VRF MAP: 139 lbs/ac\nVRF Potash: 121 lbs/ac
430,2022-11-17,5,VRF MAP: 154 lbs/ac
431,2022-11-17,5,VRF MAP: 115.8 lbs/ac


Now, we have everything ready. Let's create the final dictionary.

In [17]:
logs = []
for index, row in sub_log_df.iterrows():
    logs.append({
        "model": "acrelog.operation", 
        "pk": index+1,
        "fields": {
            "date": row['date'],
            "operator": row['operator'],
            "note": row['note'],
        }
    })
logs

[{'model': 'acrelog.operation',
  'pk': 1,
  'fields': {'date': '2022-04-24',
   'operator': 1,
   'note': '2-4D round up burn down '}},
 {'model': 'acrelog.operation',
  'pk': 2,
  'fields': {'date': '2022-04-27', 'operator': 2, 'note': '20oz ru, 11 24d'}},
 {'model': 'acrelog.operation',
  'pk': 3,
  'fields': {'date': '2022-04-27',
   'operator': 2,
   'note': '20 oz ru/ 11 oz 24d'}},
 {'model': 'acrelog.operation',
  'pk': 4,
  'fields': {'date': '2022-04-27',
   'operator': 2,
   'note': '20 oz ru. 11 oz 24d'}},
 {'model': 'acrelog.operation',
  'pk': 5,
  'fields': {'date': '2022-04-27',
   'operator': 2,
   'note': '20 oz ru 11 oz 24d'}},
 {'model': 'acrelog.operation',
  'pk': 6,
  'fields': {'date': '2022-04-27', 'operator': 1, 'note': nan}},
 {'model': 'acrelog.operation',
  'pk': 7,
  'fields': {'date': '2022-04-27', 'operator': 1, 'note': nan}},
 {'model': 'acrelog.operation',
  'pk': 8,
  'fields': {'date': '2022-04-27', 'operator': 1, 'note': nan}},
 {'model': 'acrelog.op

You can dump the dictionary to JSON then load the JSON file to the database.